In [12]:
import os
import logging
from clickhouse_driver import Client
from dotenv import load_dotenv
from datetime import datetime, timedelta, date

# Загрузка переменных окружения из файла .env
load_dotenv()

# Настройка логирования
logging.basicConfig(
    level=logging.INFO,  # Используем INFO для менее подробного логирования
    format='%(asctime)s - %(levelname)s - %(message)s',
    handlers=[
        logging.FileHandler("cyclic_algorithm.log", encoding='utf-8'),
        logging.StreamHandler()
    ]
)
logger = logging.getLogger()

# Параметры подключения к ClickHouse
CLICKHOUSE_HOST = os.getenv('CLICKHOUSE_HOST', '10.95.19.132')
CLICKHOUSE_USER = os.getenv('CLICKHOUSE_USER', 'default')
CLICKHOUSE_PASSWORD = os.getenv('CLICKHOUSE_PASSWORD', 'quie1ahpoo5Su0wohpaedae8keeph6bi')
CLICKHOUSE_DB = os.getenv('CLICKHOUSE_DB', 'default')

# Создание клиента ClickHouse
client = Client(
    host=CLICKHOUSE_HOST,
    user=CLICKHOUSE_USER,
    password=CLICKHOUSE_PASSWORD,
    port=9000,
    secure=False,
    settings={'strings_encoding': 'utf-8'}
)

##########################
# Шаг I: Расчет и обновление Status_P
##########################

def get_initial_date():
    """
    Получает самую раннюю дату из таблицы OlapCube_VNV.
    """
    try:
        query = f"""
        SELECT MIN(Dates) AS initial_date
        FROM {CLICKHOUSE_DB}.OlapCube_VNV
        """
        result = client.execute(query)
        initial_date = result[0][0]
        logger.info(f"Начальная дата в OlapCube_VNV: {initial_date}")
        return initial_date
    except Exception as e:
        logger.error(f"Ошибка при получении начальной даты: {e}", exc_info=True)
        return None

def get_target_dates(initial_date):
    """
    Вычисляет две последовательные target_dates на основе initial_date.
    
    :param initial_date: Начальная дата (datetime.date)
    :return: Список из двух дат [target_date1, target_date2]
    """
    try:
        target_date1 = initial_date + timedelta(days=1)
        target_date2 = initial_date + timedelta(days=2)
        logger.info(f"Целевые даты: {target_date1}, {target_date2}")
        return [target_date1, target_date2]
    except Exception as e:
        logger.error(f"Ошибка при вычислении target_dates: {e}", exc_info=True)
        return []

def fetch_previous_and_target_data(target_date):
    """
    Извлекает данные для target_date -1 и target_date из OlapCube_VNV.
    Также извлекает текущее значение Status_P и Status для target_date.
    
    :param target_date: Дата, для которой выполняется обновление (datetime.date)
    :return: Кортеж из двух словарей (prev_dict, target_dict)
    """
    try:
        previous_date = target_date - timedelta(days=1)
        logger.info(f"Обработка предыдущей даты: {previous_date}")
        logger.info(f"Обработка target_date: {target_date}")

        # Извлечение данных за предыдущую дату
        query_prev = f"""
        SELECT 
            serialno, Status_P, sne, ppr, ll, oh, BR, daily_flight_hours, repair_days, RepairTime, ac_typ
        FROM {CLICKHOUSE_DB}.OlapCube_VNV
        WHERE Dates = '{previous_date}'
        """
        prev_data = client.execute(query_prev)
        logger.info(f"Извлечено {len(prev_data)} записей за предыдущую дату {previous_date}.")

        # Извлечение данных за target_date, включая текущее значение Status_P и Status
        query_target = f"""
        SELECT 
            serialno, Status_P, sne, ppr, ll, oh, BR, daily_flight_hours, repair_days, RepairTime, ac_typ, Status
        FROM {CLICKHOUSE_DB}.OlapCube_VNV
        WHERE Dates = '{target_date}'
        """
        target_data = client.execute(query_target)
        logger.info(f"Извлечено {len(target_data)} записей за target_date {target_date}.")

        # Преобразование данных в словари для быстрого доступа по serialno
        prev_dict = {row[0]: row for row in prev_data}
        target_dict = {row[0]: row for row in target_data}

        return prev_dict, target_dict
    except Exception as e:
        logger.error(f"Ошибка при извлечении данных: {e}", exc_info=True)
        return None, None

def calculate_status_p(prev_dict, target_dict, target_date):
    """
    Рассчитывает Status_P для target_date на основе предыдущих данных и условий.
    Возвращает список обновлений: (Status_P, serialno, target_date)
    """
    status_updates = []
    unresolved = []
    changed_serialnos = []

    for serialno, target_row in target_dict.items():
        target_status_p = target_row[1]  # Текущее значение Status_P
        target_ll = target_row[2]
        target_oh = target_row[3]
        target_BR = target_row[4]
        target_daily_flight_hours = target_row[5]

        prev_row = prev_dict.get(serialno, None)
        if not prev_row:
            logger.warning(f"Не найдены данные за предыдущую дату для serialno: {serialno}")
            unresolved.append(serialno)
            continue

        prev_status = prev_row[1]
        prev_sne = prev_row[2]
        prev_ppr = prev_row[3]
        prev_ll = prev_row[4]
        prev_oh = prev_row[5]
        prev_BR = prev_row[6]
        prev_daily_flight_hours = prev_row[7]
        prev_repair_days = prev_row[8]
        repair_time = prev_row[9]

        # Проверка на None
        if None in [prev_sne, prev_ppr, target_ll, target_oh, target_BR, target_daily_flight_hours]:
            logger.warning(f"Пропускаем запись с serialno: {serialno} из-за отсутствия данных.")
            unresolved.append(serialno)
            continue

        # Приведение типов
        try:
            prev_sne = float(prev_sne)
            prev_ppr = float(prev_ppr)
            target_ll = float(target_ll)
            target_oh = float(target_oh)
            target_BR = float(target_BR)
            target_daily_flight_hours = float(target_daily_flight_hours)
            prev_repair_days = float(prev_repair_days) if prev_repair_days is not None else 0.0
            repair_time = float(repair_time) if repair_time is not None else 0.0
        except ValueError as ve:
            logger.warning(f"Некорректные числовые значения для serialno: {serialno}. Ошибка: {ve}")
            unresolved.append(serialno)
            continue

        # Инициализация Status_P
        status_p = None

        # 1. Эксплуатация
        if prev_status == "Эксплуатация":
            condition_sne = prev_sne < (target_ll - target_daily_flight_hours)
            condition_ppr = prev_ppr < (target_oh - target_daily_flight_hours)

            if condition_sne and condition_ppr:
                status_p = "Эксплуатация"
            else:
                if prev_sne < target_BR:
                    status_p = "Ремонт"
                else:
                    status_p = "Хранение"

        # 2. Ремонт
        elif prev_status == "Ремонт":
            if prev_repair_days < repair_time:
                status_p = "Ремонт"
            else:
                status_p = "Исправен"

        # 3. Исправен, Неактивно, Хранение
        elif prev_status in ["Исправен", "Неактивно", "Хранение"]:
            status_p = prev_status

        else:
            unresolved.append(serialno)
            logger.warning(f"Неизвестный Status_P для serialno: {serialno} с предыдущим статусом: {prev_status}")
            continue

        if status_p:
            # Добавляем все статусы для обновления, независимо от изменений
            status_updates.append((status_p, serialno, target_date))
            # Логируем только изменения статусов
            old_status_p_clean = prev_status.strip().lower() if isinstance(prev_status, str) else prev_status
            new_status_p_clean = status_p.strip().lower() if isinstance(status_p, str) else status_p
            if old_status_p_clean != new_status_p_clean:
                changed_serialnos.append(serialno)
                logger.info(f"serialno: {serialno} - Status_P изменился с '{prev_status}' на '{status_p}'")
        else:
            unresolved.append(serialno)
            logger.warning(f"Не удалось определить Status_P для serialno: {serialno}")

    if unresolved:
        logger.error(f"Не удалось определить Status_P для следующих serialno: {unresolved}")

    logger.info(f"Определено {len(status_updates)} записей для обновления Status_P.")
    if changed_serialnos:
        logger.info(f"Изменились статусы для следующих serialno: {', '.join(changed_serialnos)}")
        logger.info(f"Общее количество измененных serialno: {len(changed_serialnos)}")
    else:
        logger.info("Изменения статусов не зафиксированы.")

    return status_updates

def update_status_p(status_updates, target_date):
    """
    Обновляет поле Status_P в таблице OlapCube_VNV для target_date.
    """
    try:
        for status_p, serialno, date in status_updates:
            update_query = f"""
            ALTER TABLE {CLICKHOUSE_DB}.OlapCube_VNV 
            UPDATE Status_P = %(status_p)s 
            WHERE serialno = %(serialno)s AND Dates = %(date)s
            """
            params = {
                'status_p': status_p,
                'serialno': serialno,
                'date': date
            }
            try:
                client.execute(update_query, params)
            except Exception as e_inner:
                logger.error(f"Ошибка при обновлении Status_P для serialno: {serialno}, Dates: {date}. Ошибка: {e_inner}", exc_info=True)
                continue  # Продолжаем обработку остальных записей
        logger.info(f"Обновлено {len(status_updates)} записей в OlapCube_VNV для даты {target_date}.")
    except Exception as e:
        logger.error(f"Ошибка при обновлении Status_P: {e}", exc_info=True)

##########################
# Шаг II: Расчет и обновление балансов
##########################

def calculate_and_update_balances(target_date):
    """
    Рассчитывает и обновляет balance_mi8t, balance_mi17, balance_total, 
    stock_mi8t, stock_mi17, stock_total для target_date в таблице OlapCube_VNV.
    """
    try:
        # 1. Рассчитать balance_mi8t
        query_balance_mi8t = f"""
        SELECT COUNT(*) 
        FROM {CLICKHOUSE_DB}.OlapCube_VNV
        WHERE ac_typ = 'Ми-8Т' AND Status_P = 'Эксплуатация' AND Dates = '{target_date}'
        """
        balance_mi8t_count = client.execute(query_balance_mi8t)[0][0]
        logger.info(f"balance_mi8t_count: {balance_mi8t_count}")

        # Получить mi8t_count
        query_mi8t_count = f"""
        SELECT mi8t_count 
        FROM {CLICKHOUSE_DB}.OlapCube_VNV
        WHERE Dates = '{target_date}'
        LIMIT 1
        """
        mi8t_count_result = client.execute(query_mi8t_count)
        if mi8t_count_result:
            mi8t_count = mi8t_count_result[0][0]
            balance_mi8t = balance_mi8t_count - mi8t_count
            logger.info(f"balance_mi8t: {balance_mi8t} (balance_mi8t_count: {balance_mi8t_count} - mi8t_count: {mi8t_count})")
        else:
            logger.warning(f"mi8t_count не найден для даты {target_date}. Устанавливаем balance_mi8t = balance_mi8t_count.")
            balance_mi8t = balance_mi8t_count

        # 2. Рассчитать balance_mi17
        query_balance_mi17 = f"""
        SELECT COUNT(*) 
        FROM {CLICKHOUSE_DB}.OlapCube_VNV
        WHERE ac_typ = 'Ми-17' AND Status_P = 'Эксплуатация' AND Dates = '{target_date}'
        """
        balance_mi17_count = client.execute(query_balance_mi17)[0][0]
        logger.info(f"balance_mi17_count: {balance_mi17_count}")

        # Получить mi17_count
        query_mi17_count = f"""
        SELECT mi17_count 
        FROM {CLICKHOUSE_DB}.OlapCube_VNV
        WHERE Dates = '{target_date}'
        LIMIT 1
        """
        mi17_count_result = client.execute(query_mi17_count)
        if mi17_count_result:
            mi17_count = mi17_count_result[0][0]
            balance_mi17 = balance_mi17_count - mi17_count
            logger.info(f"balance_mi17: {balance_mi17} (balance_mi17_count: {balance_mi17_count} - mi17_count: {mi17_count})")
        else:
            logger.warning(f"mi17_count не найден для даты {target_date}. Устанавливаем balance_mi17 = balance_mi17_count.")
            balance_mi17 = balance_mi17_count

        # 3. Рассчитать balance_total
        balance_total = balance_mi8t + balance_mi17
        logger.info(f"balance_total: {balance_total} (balance_mi8t: {balance_mi8t} + balance_mi17: {balance_mi17})")

        # 4. Рассчитать stock_mi8t
        query_stock_mi8t = f"""
        SELECT COUNT(*) 
        FROM {CLICKHOUSE_DB}.OlapCube_VNV
        WHERE ac_typ = 'Ми-8Т' AND Status_P = 'Исправен' AND Dates = '{target_date}'
        """
        stock_mi8t = client.execute(query_stock_mi8t)[0][0]
        logger.info(f"stock_mi8t: {stock_mi8t}")

        # 5. Рассчитать stock_mi17
        query_stock_mi17 = f"""
        SELECT COUNT(*) 
        FROM {CLICKHOUSE_DB}.OlapCube_VNV
        WHERE ac_typ = 'Ми-17' AND Status_P = 'Исправен' AND Dates = '{target_date}'
        """
        stock_mi17 = client.execute(query_stock_mi17)[0][0]
        logger.info(f"stock_mi17: {stock_mi17}")

        # 6. Рассчитать stock_empty (ac_typ пусто и Status_P = 'Исправен')
        query_stock_empty = f"""
        SELECT COUNT(*) 
        FROM {CLICKHOUSE_DB}.OlapCube_VNV
        WHERE (ac_typ = '' OR ac_typ IS NULL) AND Status_P = 'Исправен' AND Dates = '{target_date}'
        """
        stock_empty = client.execute(query_stock_empty)[0][0]
        logger.info(f"stock_empty: {stock_empty}")

        # 7. Рассчитать stock_total
        stock_total = stock_mi8t + stock_mi17 + stock_empty
        logger.info(f"stock_total: {stock_total} (stock_mi8t: {stock_mi8t} + stock_mi17: {stock_mi17} + stock_empty: {stock_empty})")

        # 8. Обновить таблицу OlapCube_VNV
        update_query = f"""
        ALTER TABLE {CLICKHOUSE_DB}.OlapCube_VNV 
        UPDATE 
            balance_mi8t = {balance_mi8t},
            balance_mi17 = {balance_mi17},
            balance_total = {balance_total},
            stock_mi8t = {stock_mi8t},
            stock_mi17 = {stock_mi17},
            stock_total = {stock_total}
        WHERE Dates = '{target_date}'
        """
        client.execute(update_query)
        logger.info(f"Баланс и запасы успешно обновлены для даты {target_date}.")

    ##########################
    # Шаг III: Корректировка Status
    ##########################

def adjust_status(balance_total, target_date):
    """
    Корректирует поле Status на основе balance_total.
    
    :param balance_total: Значение balance_total
    :param target_date: Дата, для которой выполняется корректировка
    """
    try:
        if balance_total == 0:
            # Копируем Status_P в Status
            update_query = f"""
            ALTER TABLE {CLICKHOUSE_DB}.OlapCube_VNV 
            UPDATE Status = Status_P 
            WHERE Dates = '{target_date}'
            """
            client.execute(update_query)
            logger.info(f"balance_total = 0: Все значения Status_P скопированы в Status для даты {target_date}.")

        elif balance_total > 0:
            # balance_total > 0: Меняем первые balance_total записей с Status_P='Эксплуатация' на Status='Исправен'
            select_query = f"""
            SELECT serialno 
            FROM {CLICKHOUSE_DB}.OlapCube_VNV 
            WHERE Dates = '{target_date}' AND Status_P = 'Эксплуатация' 
            LIMIT {int(balance_total)}
            """
            records = client.execute(select_query)
            serialnos = [row[0] for row in records]

            if serialnos:
                # Обновление всех выбранных записей в одном запросе
                update_query = f"""
                ALTER TABLE {CLICKHOUSE_DB}.OlapCube_VNV 
                UPDATE Status = 'Исправен' 
                WHERE serialno IN %(serialnos)s AND Dates = %(date)s
                """
                params = {
                    'serialnos': serialnos,
                    'date': target_date
                }
                client.execute(update_query, params)
                logger.info(f"Status изменен на 'Исправен' для {len(serialnos)} записей.")

        elif balance_total < 0:
            # balance_total < 0: Меняем abs(balance_total) записей
            required_changes = abs(int(balance_total))

            # Сначала выбираем записи с Status_P='Исправен'
            query_inspect = f"""
            SELECT serialno 
            FROM {CLICKHOUSE_DB}.OlapCube_VNV 
            WHERE Dates = '{target_date}' AND Status_P = 'Исправен' 
            LIMIT {required_changes}
            """
            records = client.execute(query_inspect)
            serialnos = [row[0] for row in records]
            changes_made = len(serialnos)

            if serialnos:
                # Обновление всех выбранных записей в одном запросе
                update_query = f"""
                ALTER TABLE {CLICKHOUSE_DB}.OlapCube_VNV 
                UPDATE Status = 'Эксплуатация' 
                WHERE serialno IN %(serialnos)s AND Dates = %(date)s
                """
                params = {
                    'serialnos': serialnos,
                    'date': target_date
                }
                client.execute(update_query, params)
                logger.info(f"Status изменен на 'Эксплуатация' для {len(serialnos)} записей.")

            # Если недостаточно, берем из 'Неактивно'
            if changes_made < required_changes:
                remaining = required_changes - changes_made
                query_inspect = f"""
                SELECT serialno 
                FROM {CLICKHOUSE_DB}.OlapCube_VNV 
                WHERE Dates = '{target_date}' AND Status_P = 'Неактивно' 
                LIMIT {remaining}
                """
                records = client.execute(query_inspect)
                serialnos = [row[0] for row in records]

                if serialnos:
                    update_query = f"""
                    ALTER TABLE {CLICKHOUSE_DB}.OlapCube_VNV 
                    UPDATE Status = 'Эксплуатация' 
                    WHERE serialno IN %(serialnos)s AND Dates = %(date)s
                    """
                    params = {
                        'serialnos': serialnos,
                        'date': target_date
                    }
                    client.execute(update_query, params)
                    logger.info(f"Status изменен на 'Эксплуатация' для дополнительных {len(serialnos)} записей.")

        logger.info(f"Корректировка Status завершена для balance_total = {balance_total}.")
    except Exception as e:
        logger.error(f"Ошибка при корректировке Status на основе balance_total: {e}", exc_info=True)

##########################
# Шаг IV: Обновление счетчиков
##########################

def update_counters(prev_dict, target_dict, target_date):
    """
    Обновляет счетчики sne, ppr и repair_days в таблице OlapCube_VNV на target_date согласно заданной логике.
    
    :param prev_dict: Словарь с данными за предыдущую дату, ключ = serialno
    :param target_dict: Словарь с данными за target_date, ключ = serialno
    :param target_date: Дата, для которой выполняется обновление (datetime.date)
    """
    counter_updates = []
    unresolved = []
    total_updates = 0  # Счетчик успешных обновлений

    for serialno, target_row in target_dict.items():
        current_status = target_row[11]  # Поле Status (индекс 11)
        current_daily_flight_hours = target_row[7]  # Поле daily_flight_hours (индекс 7)

        prev_row = prev_dict.get(serialno, None)
        if not prev_row:
            unresolved.append(serialno)
            continue

        prev_status_p = prev_row[1]  # Status_P на предыдущую дату (индекс 1)
        prev_sne = prev_row[2]       # sne на предыдущую дату (индекс 2)
        prev_ppr = prev_row[3]       # ppr на предыдущую дату (индекс 3)
        prev_repair_days = prev_row[8]  # repair_days на предыдущую дату (индекс 8)

        # Приведение типов
        try:
            prev_sne = float(prev_sne) if prev_sne is not None else 0.0
            prev_ppr = float(prev_ppr) if prev_ppr is not None else 0.0
            # repair_days может быть NULL, оставляем как есть
            daily_flight_hours = float(current_daily_flight_hours) if current_daily_flight_hours is not None else 0.0
        except ValueError:
            unresolved.append(serialno)
            continue

        # Инициализация новых значений
        new_sne = prev_sne
        new_ppr = prev_ppr
        new_repair_days = prev_repair_days  # По умолчанию

        if current_status == "Эксплуатация":
            new_sne += daily_flight_hours
            new_ppr += daily_flight_hours

        elif current_status == "Исправен":
            if prev_status_p == "Ремонт":
                new_ppr = 0.0
                new_repair_days = None  # Устанавливаем в NULL
            else:
                new_ppr = prev_ppr

        elif current_status == "Ремонт":
            if prev_status_p == "Эксплуатация":
                new_repair_days = 1.0
            elif prev_repair_days is not None:
                new_repair_days += 1.0
            else:
                new_repair_days = 1.0  # Если предыдущие repair_days были NULL

        elif current_status in ["Хранение", "Неактивно"]:
            # sne и ppr остаются без изменений
            pass

        else:
            unresolved.append(serialno)
            continue

        # Округление до двух знаков после запятой
        new_sne = round(new_sne, 2)
        new_ppr = round(new_ppr, 2)
        if new_repair_days is not None:
            new_repair_days = round(new_repair_days, 2)

        # Добавляем обновленные значения в список
        counter_updates.append({
            'serialno': serialno,
            'sne': new_sne,
            'ppr': new_ppr,
            'repair_days': new_repair_days,
            'date': target_date
        })

    # Выполнение обновлений
    try:
        for update in counter_updates:
            serialno = update['serialno']
            sne = update['sne']
            ppr = update['ppr']
            repair_days = update['repair_days']
            date_val = update['date']

            # Формирование SQL-запроса с учетом возможности NULL для repair_days
            if repair_days is None:
                update_query = f"""
                ALTER TABLE {CLICKHOUSE_DB}.OlapCube_VNV 
                UPDATE sne = %(sne)s, ppr = %(ppr)s, repair_days = NULL 
                WHERE serialno = %(serialno)s AND Dates = %(date)s
                """
                params = {
                    'sne': sne,
                    'ppr': ppr,
                    'serialno': serialno,
                    'date': date_val
                }
            else:
                update_query = f"""
                ALTER TABLE {CLICKHOUSE_DB}.OlapCube_VNV 
                UPDATE sne = %(sne)s, ppr = %(ppr)s, repair_days = %(repair_days)s 
                WHERE serialno = %(serialno)s AND Dates = %(date)s
                """
                params = {
                    'sne': sne,
                    'ppr': ppr,
                    'repair_days': repair_days,
                    'serialno': serialno,
                    'date': date_val
                }

            try:
                client.execute(update_query, params)
                total_updates += 1
            except Exception as e_inner:
                # Ошибки отдельных обновлений не прерывают процесс
                unresolved.append(serialno)
                logger.error(f"Ошибка при обновлении счетчиков для serialno: {serialno}, Dates: {date_val}. Ошибка: {e_inner}", exc_info=True)
                continue

        # Логирование общего количества изменений
        logger.info(f"Обновлено {total_updates} записей счетчиков sne, ppr и repair_days для даты {target_date}.")
        if unresolved:
            logger.warning(f"Не удалось обновить счетчики для следующих serialno: {', '.join(unresolved)}")
    except Exception as e:
        logger.error(f"Ошибка при обновлении счетчиков: {e}", exc_info=True)

##########################
# Интеграция всех шагов в циклический алгоритм
##########################

def main():
    """
    Главная функция, выполняющая циклический алгоритм со всеми четырьмя шагами для двух дат.
    """
    # Шаг I: Получение начальной даты
    initial_date = get_initial_date()
    if not initial_date:
        logger.error("Не удалось определить начальную дату. Завершаем выполнение.")
        return

    # Преобразование initial_date в объект date, если это строка или datetime
    if isinstance(initial_date, str):
        try:
            initial_date_date = datetime.strptime(initial_date, "%Y-%m-%d").date()
        except ValueError as ve:
            logger.error(f"Неверный формат initial_date: {initial_date}. Ошибка: {ve}", exc_info=True)
            return
    elif isinstance(initial_date, datetime):
        initial_date_date = initial_date.date()
    elif isinstance(initial_date, date):
        initial_date_date = initial_date
    else:
        logger.error(f"Неизвестный формат initial_date: {initial_date}")
        return

    # Шаг I: Вычисление двух target_dates
    target_dates = get_target_dates(initial_date_date)
    if not target_dates:
        logger.error("Не удалось определить target_dates. Завершаем выполнение.")
        return

    target_date1, target_date2 = target_dates

    ##########################
    # Обработка первой target_date (target_date1)
    ##########################
    logger.info(f"Начало обработки первой target_date: {target_date1}")

    # Шаг I: Извлечение данных за предыдущую и target_date1
    prev_dict1, target_dict1 = fetch_previous_and_target_data(target_date1)
    if not prev_dict1 or not target_dict1:
        logger.error(f"Не удалось извлечь необходимые данные для даты {target_date1}. Пропускаем обработку этой даты.")
    else:
        # Шаг I: Расчет Status_P и обновление
        status_updates1 = calculate_status_p(prev_dict1, target_dict1, target_date1)
        if status_updates1:
            update_status_p(status_updates1, target_date1)
        else:
            logger.info(f"Нет записей для обновления Status_P для даты {target_date1}.")

        # Шаг II: Расчет и обновление балансов
        calculate_and_update_balances(target_date1)

        # Шаг III: Корректировка Status на основе balance_total
        try:
            # Получить balance_total для target_date1
            query_balance_total = f"""
            SELECT balance_total
            FROM {CLICKHOUSE_DB}.OlapCube_VNV
            WHERE Dates = '{target_date1}'
            LIMIT 1
            """
            balance_total_result = client.execute(query_balance_total)
            if balance_total_result and balance_total_result[0][0] is not None:
                balance_total1 = balance_total_result[0][0]
                logger.info(f"balance_total на target_date1: {balance_total1}")
                adjust_status(balance_total1, target_date1)
            else:
                logger.error(f"balance_total не найден для даты {target_date1}. Пропускаем корректировку Status.")
        except Exception as e:
            logger.error(f"Ошибка при получении balance_total для даты {target_date1}: {e}", exc_info=True)

        # Шаг IV: Обновление счетчиков для target_date1
        update_counters(prev_dict1, target_dict1, target_date1)

    ##########################
    # Обработка второй target_date (target_date2)
    ##########################
    logger.info(f"Начало обработки второй target_date: {target_date2}")

    # Шаг I: Извлечение данных за предыдущую и target_date2
    prev_dict2, target_dict2 = fetch_previous_and_target_data(target_date2)
    if not prev_dict2 or not target_dict2:
        logger.error(f"Не удалось извлечь необходимые данные для даты {target_date2}. Пропускаем обработку этой даты.")
    else:
        # Шаг I: Расчет Status_P и обновление
        status_updates2 = calculate_status_p(prev_dict2, target_dict2, target_date2)
        if status_updates2:
            update_status_p(status_updates2, target_date2)
        else:
            logger.info(f"Нет записей для обновления Status_P для даты {target_date2}.")

        # Шаг IV: Обновление счетчиков для target_date2
        update_counters(prev_dict2, target_dict2, target_date2)

    logger.info("Все необходимые даты обработаны.")

if __name__ == "__main__":
    main()


SyntaxError: expected 'except' or 'finally' block (1103103231.py, line 365)